# Se importan las bibliotecas necesarias

In [1]:
import pandas as pd
import re
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from catboost import CatBoostRegressor
%matplotlib inline

# Se lee el .csv

In [2]:
train = pd.read_csv('csv/train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


# Elaboración del modelo

Para este modelo se utilizarán árboles de decisión para realizar las predicciones y se tendrán en cuenta los siguientes features extraídos del análisis exploratorio, los cuales fueron seleccionados por el grupo como aquellos que podrían resultan más interesantes:

- Palabras con mayor y menor porcentaje de veracidad.
- Pares de 2 palabras con mayor y menor porcentaje de veracidad.
- Tweets que contienen @, ¿?, ¡! tienden a ser falsos.
- Keywords con mayor y menor porcentaje de veracidad.
- Locaciones más y menos veraces.
- Longitud.

### Preparación del set de datos

#### Parte inicial

In [4]:
#Palabras con mayor y menor porcentaje de veracidad
min_repetitions = ((0.2/100)*len(train.index))
words = train['text'].str.split()
clean_words = []

for sentence in words:
    clean_sentence = []
    for word in sentence:
        clean_word = re.sub('[^A-Za-z0-9]+','', word)
        if(clean_word != ''):
            clean_sentence.append(clean_word.lower())
    clean_words.append(clean_sentence)

In [5]:
train_aux = train
train_aux['words'] = clean_words
train_aux = train_aux.explode('words')
train_aux.head()

,id,keyword,location,text,target,words
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,are
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,the
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,reason


In [6]:
train_aux = train_aux.groupby('words').agg({'target':['sum','count']})
train_aux = train_aux[train_aux['target']['count']>min_repetitions]
train_aux['veracity'] = train_aux['target']['sum']/train_aux['target']['count']
train_aux.head()

target        veracity
         sum count          
words                       
05        14    18  0.777778
1         29    64  0.453125
10         9    23  0.391304
100        6    17  0.352941
11         7    21  0.333333

In [7]:
train_aux.reset_index(inplace=True) 

In [8]:
#Pares de 2 palabras con mayor y menor porcentaje de veracidad.
train_aux2 = train
train_aux2['words'] = clean_words
words_pairs = []

for sentence in train_aux2['words']:
    pairs = []
    for i in range(len(sentence)-1):
        pairs.append(sentence[i] + ' ' + sentence[i+1])
    words_pairs.append(pairs)
    
train_aux2['words'] = words_pairs
train_aux2 = train_aux2.explode('words')
train_aux2.head()

,id,keyword,location,text,target,words
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds are
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,are the
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,the reason
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,reason of


In [9]:
train_aux2 = train_aux2.groupby('words').agg({'target':['sum','count']})
train_aux2 = train_aux2[train_aux2['target']['count']>min_repetitions]
train_aux2['veracity'] = train_aux2['target']['sum']/train_aux2['target']['count']

In [10]:
#las palabras que son '' la cagan
train_aux2.head(10)

target        veracity
                 sum count          
words                               
11yearold boy     20    20  1.000000
16yr old          28    28  1.000000
40 families       26    26  1.000000
70 years          30    31  0.967742
a blast            3    16  0.187500
a car              9    16  0.562500
a cliff            8    24  0.333333
a few              8    18  0.444444
a fire             8    16  0.500000
a good             5    16  0.312500

In [11]:
train_aux2.reset_index(inplace=True)

In [12]:
# Keywords más y menos veraces
train_aux3 = train
train_aux3 = train_aux3.groupby('keyword').agg({'target':['sum','count']})
train_aux3 = train_aux3[train_aux3['target']['count']>min_repetitions]
train_aux3['veracity'] = train_aux3['target']['sum']/train_aux3['target']['count']
train_aux3.head()

target        veracity
                       sum count          
keyword                                   
ablaze                  13    36  0.361111
accident                24    35  0.685714
aftershock               0    34  0.000000
airplane%20accident     30    35  0.857143
ambulance               20    38  0.526316

In [13]:
train_aux3.reset_index(inplace=True)

In [14]:
# Locaciones más y menos veraces
train_aux4 = train
train_aux4 = train_aux4.groupby('location').agg({'target':['sum','count']})
train_aux4 = train_aux4[train_aux4['target']['count']>min_repetitions]
train_aux4['veracity'] = train_aux4['target']['sum']/train_aux4['target']['count']
train_aux4.head()

target        veracity
               sum count          
location                          
Australia        9    18  0.500000
California       7    17  0.411765
Canada          13    29  0.448276
Chicago, IL      9    18  0.500000
India           20    24  0.833333

In [15]:
train_aux4.reset_index(inplace=True)

#### Parte final

In [16]:
top_words = train_aux[train_aux['veracity']>0.9]['words']
worst_words = train_aux[train_aux['veracity']<0.1]['words']

In [17]:
top_words_pairs = train_aux2[train_aux2['veracity']>0.9]['words']
worst_words_pairs = train_aux2[train_aux2['veracity']<0.1]['words']

In [18]:
top_keywords = train_aux3[train_aux3['veracity']>0.9]['keyword']
worst_keywords = train_aux3[train_aux3['veracity']<0.1]['keyword']

In [19]:
top_locations = train_aux4[train_aux4['veracity']>0.9]['location']
worst_locations = train_aux4[train_aux4['veracity']<0.1]['location']

In [20]:
train = train.drop(columns='words')
train['long(char)'] = train['text'].str.len()

aux = []
for i in words:
    aux.append(len(i))

train['long(word)'] = aux
train.head()

,id,keyword,location,text,target,long(char),long(word)
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16


In [21]:
def count_text_ocurrences(texts, words):
    ocurrences = []
    for text in texts:
        count = 0
        for word in words:
            if word in text:
                count = count+1
        ocurrences.append(count)
    return ocurrences

def count_ocurrences(searched, series):
    ocurrences = []
    for element in searched:
        count = 0
        if element in series:
            count = count+1
        ocurrences.append(count)
    return ocurrences

def contains_char(texts, char1, char2=''):
    ocurrences = []
    for text in texts:
        count = 0
        if char1 in text:
            count = 1
        if (count==0)&(char2!=''):
            if char2 in text:
                count = 1
        ocurrences.append(count)
    return ocurrences

In [22]:
train['top_words'] = count_text_ocurrences(train['text'], top_words)
train['worst_words'] = count_text_ocurrences(train['text'], worst_words)
train['top_words_pairs'] = count_text_ocurrences(train['text'], top_words_pairs)
train['worst_words_pairs'] = count_text_ocurrences(train['text'], worst_words_pairs)
train.head()

,id,keyword,location,text,target,long(char),long(word),top_words,worst_words,top_words_pairs,worst_words_pairs
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,1,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0


In [23]:
train['top_keywords'] = count_ocurrences(train['keyword'], top_keywords)
train['worst_keywords'] = count_ocurrences(train['keyword'], worst_keywords)
train['top_locations'] = count_ocurrences(train['location'], top_locations)
train['worst_locations'] = count_ocurrences(train['location'], worst_locations)
train.head()

,id,keyword,location,text,target,long(char),long(word),top_words,worst_words,top_words_pairs,worst_words_pairs,top_keywords,worst_keywords,top_locations,worst_locations
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,0,0,0,0,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,1,0,0,0,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,0,0,0


In [24]:
train['arroba'] = contains_char(train['text'],'@')
train['singnosDeInterrogacion'] = contains_char(train['text'],'\?', '¿')
train['signosDeExclamacion'] = contains_char(train['text'],'!','¡')
train.head()

,id,keyword,location,text,target,long(char),long(word),top_words,worst_words,top_words_pairs,worst_words_pairs,top_keywords,worst_keywords,top_locations,worst_locations,arroba,singnosDeInterrogacion,signosDeExclamacion
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,0,0,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,0,0,0,0,0,0,0,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,1,0,0,0,0,0,0,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,0,0,0,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,0,0,0,0,0,0


In [25]:
train = train.drop(columns=['keyword','location','text'])
train = train.set_index('id')
train.head(20)

,target,long(char),long(word),top_words,worst_words,top_words_pairs,worst_words_pairs,top_keywords,worst_keywords,top_locations,worst_locations,arroba,singnosDeInterrogacion,signosDeExclamacion
id,,,,,,,,,,,,,,
1,1,69,13,0,0,0,0,0,0,0,0,0,0,0
4,1,38,7,0,0,0,0,0,0,0,0,0,0,0
5,1,133,22,1,0,0,0,0,0,0,0,0,0,0
6,1,65,8,1,0,0,0,0,0,0,0,0,0,0
7,1,88,16,1,0,0,0,0,0,0,0,0,0,0
8,1,110,18,1,0,0,0,0,0,0,0,0,0,0
10,1,95,14,0,0,0,0,0,0,0,0,0,0,0
13,1,59,15,0,0,1,0,0,0,0,0,0,0,0
14,1,79,12,0,0,0,0,0,0,0,0,0,0,0


In [26]:
col_rename = {'long(char)': 'longEnChars', 'long(word)': 'longEnWords', 'top_words': 'topWords', 'worst_words': 'worstWords', 'top_words_pairs': 'topWordsPairs',  
             'worst_words_pairs': 'worstWordsPairs', 'top_keywords': 'topKeywords', 'worst_keywords': 'worstKeywords', 'top_locations': 'topLocations', 
             'worst_locations': 'worstLocations'}
train = train.rename(columns= col_rename)
train.head(20)

,target,longEnChars,longEnWords,topWords,worstWords,topWordsPairs,worstWordsPairs,topKeywords,worstKeywords,topLocations,worstLocations,arroba,singnosDeInterrogacion,signosDeExclamacion
id,,,,,,,,,,,,,,
1,1,69,13,0,0,0,0,0,0,0,0,0,0,0
4,1,38,7,0,0,0,0,0,0,0,0,0,0,0
5,1,133,22,1,0,0,0,0,0,0,0,0,0,0
6,1,65,8,1,0,0,0,0,0,0,0,0,0,0
7,1,88,16,1,0,0,0,0,0,0,0,0,0,0
8,1,110,18,1,0,0,0,0,0,0,0,0,0,0
10,1,95,14,0,0,0,0,0,0,0,0,0,0,0
13,1,59,15,0,0,1,0,0,0,0,0,0,0,0
14,1,79,12,0,0,0,0,0,0,0,0,0,0,0


### Entrenamiento del set de datos

In [27]:
X, y = train.iloc[:,1:-1], train.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=465)

#### XGBoost

In [28]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 20, alpha = 10, n_estimators = 80)

In [29]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=20,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=80, n_jobs=0, num_parallel_tree=1,
             objective='binary:logistic', random_state=0, reg_alpha=10,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [30]:
preds = xg_reg.predict(X_test)

In [31]:
print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Accuracy Score : 0.680385288966725
Precision Score : 0.6779220779220779
Recall Score : 0.5199203187250996
F1 Score : 0.5885005636978581


In [32]:
df = pd.DataFrame(preds)
final = df.to_csv('csv/xgboost.csv')
df.head()

,0
0,0.246681
1,0.244642
2,0.474116
3,0.539382
4,0.355631


In [33]:
# params = {
#         'min_child_weight': [1, 5, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.2, 0.3, 0.4],
#         'max_depth': [10, 15, 20]
#         }
# grid_acc = GridSearchCV(xg_reg, param_grid = params)
# grid_acc.fit(X_train, y_train)
# y_pred_acc = grid_acc.predict(X_test)

# # New Model Evaluation metrics 
# print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred_acc.round())))
# print('Precision Score : ' + str(precision_score(y_test,y_pred_acc.round())))
# print('Recall Score : ' + str(recall_score(y_test,y_pred_acc.round())))
# print('F1 Score : ' + str(f1_score(y_test,y_pred_acc.round())))

#### RandomForest

In [34]:
rf_model = RandomForestRegressor(random_state=13, n_estimators=80, max_depth=20)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Accuracy Score : 0.6663747810858144
Precision Score : 0.631236442516269
Recall Score : 0.5796812749003984
F1 Score : 0.604361370716511


In [36]:
df = pd.DataFrame(preds)
final = df.to_csv('csv/randomForest.csv')
df.head()

,0
0,0.324366
1,0.665225
2,0.340965
3,0.202070
4,0.098478


rf_model.feature_importances_
plt.figure(figsize=(17,9))
plt.bar(X_train.columns, rf_model.feature_importances_)
plt.xlabel('Features')
plt.ylabel('Importancia')
plt.title('Importancia Features con RF')
plt.show()

#### LightGBM

In [37]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
     'objective': 'regression',
     'metric': 'rmse',
     'num_leaves': 5,
     'learning_rate': 0.1,
     'feature_fraction': 0.9,
 }

params = {
     'objective': 'regression',
 }

gbm = lgb.train(params,
                 lgb_train,
                 num_boost_round=100,
                 valid_sets=lgb_eval,
                 early_stopping_rounds=10)

preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)
print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

[1]	valid_0's l2: 0.236694
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.228451
[3]	valid_0's l2: 0.221745
[4]	valid_0's l2: 0.216587
[5]	valid_0's l2: 0.21209
[6]	valid_0's l2: 0.208704
[7]	valid_0's l2: 0.205716
[8]	valid_0's l2: 0.203293
[9]	valid_0's l2: 0.201286
[10]	valid_0's l2: 0.199541
[11]	valid_0's l2: 0.198386
[12]	valid_0's l2: 0.197446
[13]	valid_0's l2: 0.19672
[14]	valid_0's l2: 0.195963
[15]	valid_0's l2: 0.195304
[16]	valid_0's l2: 0.194728
[17]	valid_0's l2: 0.19426
[18]	valid_0's l2: 0.193871
[19]	valid_0's l2: 0.19343
[20]	valid_0's l2: 0.193031
[21]	valid_0's l2: 0.192723
[22]	valid_0's l2: 0.192867
[23]	valid_0's l2: 0.192569
[24]	valid_0's l2: 0.192365
[25]	valid_0's l2: 0.192197
[26]	valid_0's l2: 0.191744
[27]	valid_0's l2: 0.191467
[28]	valid_0's l2: 0.191369
[29]	valid_0's l2: 0.191555
[30]	valid_0's l2: 0.191342
[31]	valid_0's l2: 0.191225
[32]	valid_0's l2: 0.191267
[33]	valid_0's l2: 0.19112
[34]	valid_0's l2: 0.191061
[

In [38]:
df = pd.DataFrame(preds)
final = df.to_csv('csv/lightgbm.csv')
df.head()

,0
0,0.239668
1,0.281058
2,0.434400
3,0.601783
4,0.322696


#### CatBoost

In [39]:
model = CatBoostRegressor()
model.fit(X_train, y_train)
preds = model.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Learning rate set to 0.054419
0:	learn: 0.4903115	total: 53.3ms	remaining: 53.2s
1:	learn: 0.4863305	total: 60ms	remaining: 29.9s
2:	learn: 0.4828689	total: 65ms	remaining: 21.6s
3:	learn: 0.4798964	total: 72.8ms	remaining: 18.1s
4:	learn: 0.4769286	total: 79.4ms	remaining: 15.8s
5:	learn: 0.4740281	total: 83.8ms	remaining: 13.9s
6:	learn: 0.4714956	total: 88.8ms	remaining: 12.6s
7:	learn: 0.4695728	total: 96.3ms	remaining: 11.9s
8:	learn: 0.4677696	total: 101ms	remaining: 11.1s
9:	learn: 0.4658284	total: 106ms	remaining: 10.5s
10:	learn: 0.4639119	total: 111ms	remaining: 9.94s
11:	learn: 0.4620560	total: 117ms	remaining: 9.62s
12:	learn: 0.4605722	total: 122ms	remaining: 9.26s
13:	learn: 0.4590786	total: 126ms	remaining: 8.91s
14:	learn: 0.4576755	total: 131ms	remaining: 8.59s
15:	learn: 0.4563044	total: 135ms	remaining: 8.3s
16:	learn: 0.4552985	total: 138ms	remaining: 8s
17:	learn: 0.4543237	total: 142ms	remaining: 7.72s
18:	learn: 0.4534696	total: 145ms	remaining: 7.47s
19:	learn: 

In [40]:
df = pd.DataFrame(preds)
final = df.to_csv('csv/catboost.csv')
df.head()

,0
0,0.200908
1,0.340024
2,0.428025
3,0.540914
4,0.288130


In [41]:
model = LogisticRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Accuracy Score : 0.6882661996497373
Precision Score : 0.6901041666666666
Recall Score : 0.5278884462151394
F1 Score : 0.5981941309255079


/home/hugo/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [42]:
df = pd.DataFrame(preds)
final = df.to_csv('csv/logicRegression.csv')
df.head()

,0
0,0
1,0
2,1
3,1
4,0


In [43]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 20, alpha = 10, n_estimators = 80)
rf_model = RandomForestRegressor(random_state=13, n_estimators=80, max_depth=20)

eclf2 = VotingRegressor(estimators=[
         ('xgb', xg_reg), ('rf', rf_model)])

In [44]:
eclf2 = eclf2.fit(X, y)

In [45]:
preds = eclf2.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

Accuracy Score : 0.829246935201401
Precision Score : 0.8528735632183908
Recall Score : 0.7390438247011952
F1 Score : 0.791889007470651
